In [6]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
import pickle

In [2]:
from keras.layers.core import Dense, Dropout
from keras.models import Sequential
from keras.optimizers import nadam
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import softmax
from keras.regularizers import l2

Using TensorFlow backend.


In [59]:
path = './'

В качестве предиктора используется текст отзыва с его заголовком, для класcификации используется беггинг логистической регрессии, ранжирование проводит нейронная сеть.

In [63]:
try:
    df_test = pd.read_csv('./data_test.csv')
except:
    df_test = pd.read_csv('/data_test.csv')
    path = '/'
x_test = [x1+" "+x2 for (x1, x2) in zip(df_test['title'], df_test['text'])]

In [43]:
with open('tvc.pickle', 'rb') as f:
    """загружаем токинайзер для классификации"""
    tvс = pickle.load(f)
with open('tvr.pickle', 'rb') as f:
    """загружаем токинайзер для ранжироваания"""
    tvr = pickle.load(f)
with open('logit.pickle', 'rb') as f:
    """загружаем регрессию"""
    logit = pickle.load(f)

In [46]:
X_test_c = tvс.transform(x_test)
X_test_r = tvr.transform(x_test)

In [47]:
class_predicts = logit.predict(X_test_c)
class_predicts = pd.Series(class_predicts)
class_predicts =class_predicts.map({0: 'Ипотека', 1: 'Кредит', 2:'Реструктуризация', 3:'Вклад', 4:'Дебетовая карта', 5:'Обслуживание физ. и юр. лиц',  6:'Денежные переводы', 7:'Бизнес услуги', 8: 'Инвестиционные продукты'})

In [51]:
"""Создаем модель нейронной сети и загружаем обученные веса"""
input_shape = np.shape(X_test_r)[1]
model = Sequential()
model.add(Dense(4, input_shape=(input_shape,)))
model.add(LeakyReLU())
model.add(Dropout(0.6))

model.add(Dense(4, kernel_regularizer=l2(0.05)))
model.add(LeakyReLU())
model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))
model.load_weights('weightst_best.h5')

In [52]:
"""Берем предсказанный процент негативности"""
posneg_predicts = model.predict(X_test_r)[:,1]

In [61]:
"""Записываем результаты в csv"""
sub = pd.DataFrame({'index': range(0, len(class_predicts)),
                    'score':class_predicts})
sub.to_csv(path+'class_answer.csv', index=False)
sub = pd.DataFrame({'index': range(0, len(posneg_predicts)),
                    'type':posneg_predicts})
sub.to_csv(path+'ranking_answer.csv', index=False)